# Banda izquierda

Vamos a seleccionar al mejor interior izquierdo según:

    1. Porcentaje de pases acertados
    2. Asistencias
    3. Porcentaje de regates acertados 
    4. Goles

In [1]:
import pandas as pd
import numpy as np

#leemos el csv y eliminamos duplicidades

df = pd.read_csv('/Users/alexcar934/Desktop/EDA_bien/src/data/raw/datasetMatchInfo2020-La-Liga.csv')
df = df.drop_duplicates()



In [2]:
'''Creamos un array donde aparezcan los nombres de todos los jugadores que, 
según el dataset, al menos una vez han jugado como interior izquierdo'''
df_lm = df[df['Pos'] == 'LM']
df_lm_unique = df_lm['Player'].unique()


In [3]:
'''
Creamos un bucle for que lea todos los nombres del array para 
que les asigne la posición LM en el dataframe en todos los partidos.
'''
for i in df_lm_unique:
    if np.any(df['Player'] == i):
        df.loc[(df['Player']== i), 'Pos'] = 'LM'



/var/folders/6g/cf_j0b453g7fk51dd_1wyyzr0000gn/T/ipykernel_92885/1312128539.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Pos'][df['Player']== i] = 'LM'
/var/folders/6g/cf_j0b453g7fk51dd_1wyyzr0000gn/T/ipykernel_92885/1312128539.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Pos'][df['Player']== i] = 'LM'
/var/folders/6g/cf_j0b453g7fk51dd_1wyyzr0000gn/T/ipykernel_92885/1312128539.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

In [4]:
'''
Para segmentar, establecemos que queremos obtener únicamente los partidos
en los que el futbolista haya jugado más de 45 minutos
'''
df_lm = df[df['Pos'] == 'LM']
df_lm =df_lm[df_lm['Min']>= 45]


In [5]:
#Creamos un dataframe con los minutos jugados por jugador
#Segmentamos por los jugadores que hayan jugado, al menos, un total de 500 minutos.

df_minutos = df_lm.groupby('Player')['Min'].sum()
df_minutos_lm = df_minutos.loc[df_minutos>500].sort_values(ascending =False)


In [6]:
#Creamos el nuevo dataframe con los futbolistas que han jugado más de 500 minutos
df_lm= df_lm[df_lm['Player'].isin(df_minutos_lm.keys())]


In [7]:
#como el mínimo de minutos es de 720 (más de 8 partidos), establezco un mínimo
#de 300 toques de balón en total.
df_toques= df_lm.groupby('Player')['Performance|Touches'].sum()
df_toques_lm = df_toques.loc[df_toques>= (300)].sort_values(ascending=False)



In [8]:
#aplico el filtro al dataframe
df_lm= df_lm[df_lm['Player'].isin(df_toques_lm.keys())]


In [9]:
'''
Creamos un dataframe con la media de recuperaciones de balón por partido.
Posteriormente los ponderamos entre el total.
'''
df_recov_try= df_lm.groupby('Player')['Performance|Recov'].mean().sort_values(ascending=False)
df_recov = df_recov_try[df_recov_try.keys().isin(df_minutos_lm.keys())]
df_recov = df_recov/df_recov.sum()


In [10]:
'''
Creamos un dataframe con las asistencias realizadas por jugador.
'''
df_ast_try= df_lm.groupby('Player')['Performance|Ast'].sum().sort_values(ascending=False)
df_ast = df_ast_try[df_ast_try.keys().isin(df_minutos_lm.keys())]


In [11]:
'''
Creamos un dataframe con la media de los porcentajes de pases acertados por partido.
Posteriormente los ponderamos entre el total.
'''
df_cmp_try= df_lm.groupby('Player')['Passes|Cmp%'].mean().sort_values(ascending=False)
df_cmp= df_cmp_try[df_cmp_try.keys().isin(df_minutos_lm.keys())]
df_cmp= df_cmp/df_cmp.sum()



In [12]:
'''
Creamos un dataframe con los goles anotados por jugador.
'''
df_gls_try= df_lm.groupby('Player')['Performance|Gls'].sum().sort_values(ascending=False)
df_gls = df_gls_try[df_gls_try.keys().isin(df_minutos_lm.keys())]


In [13]:
'''
Creamos un dataframe con la media de los porcentajes de regates acertados por partido.
Posteriormente los ponderamos entre el total.
'''
df_drbls_try= df_lm.groupby('Player')['Dribbles|Succ%'].sum().sort_values(ascending=False)
df_drbls_succ = df_drbls_try[df_drbls_try.keys().isin(df_minutos_lm.keys())]
df_drbls_succ = df_drbls_succ/(df_drbls_succ.sum())


In [14]:
#Convertimos las series a dataframe
df_drbls_succ = df_drbls_succ.to_frame()
df_recov=df_recov.to_frame()
df_cmp = df_cmp.to_frame()
df_ast = df_ast.to_frame()
df_gls = df_gls.to_frame()



In [15]:
'''
Unimos todos los dataframes por la columna Player
'''
df_lm_final = df_cmp.merge(df_ast, how='inner', on=['Player'])
df_lm_final = df_lm_final.merge(df_gls, how='inner', on=['Player'])
df_lm_final =df_lm_final.merge(df_drbls_succ, how='inner', on=['Player'])


In [16]:
#añadimos la columna con los equipos de cada jugador al dataframe final
df_teams= df[['Player','Team']]
df_lm_final=df_teams.merge(df_lm_final, how='inner', on='Player').drop_duplicates()


In [17]:
#Establecemos la columna de Puntuación final
df_lm_final['Puntuación Final'] = (df_lm_final.iloc[:,2]) + (df_lm_final.iloc[:,3]*0.02) + (df_lm_final.iloc[:,4]*0.02) + (df_lm_final.iloc[:,5])
df_lm = df_lm_final.sort_values(by='Puntuación Final', ascending=False) 



In [18]:
#Cambiamos los nombres de las columnas
#Pongo iloc[1:] porque realmente Morales no es interior izquierdo, es delantero centro.
df_lm.columns =['Jugador', 'Equipo', 'Porcentaje de pases acertados', 'Asistencias', 'Goles', 'Porcentaje de regates acertados', 'Puntuación Final']
df_lm = df_lm.sort_values(by='Puntuación Final', ascending=False).iloc[1:]
df_lm

,Jugador,Equipo,Porcentaje de pases acertados,Asistencias,Goles,Porcentaje de regates acertados,Puntuación Final
258,Mikel Oyarzabal,RealS,0.035892,2.0,6.0,0.028141,0.224034
168,Javi Galán,Huesca,0.039055,2.0,0.0,0.105222,0.184277
193,Nolito,Celta,0.035806,1.0,3.0,0.055983,0.171789
239,Rubén García,Osasuna,0.032478,2.0,3.0,0.022695,0.155173
83,Fidel,Elche,0.038758,2.0,2.0,0.036302,0.155060
22,Manu Trigueros,Villarreal,0.042452,1.0,1.0,0.059006,0.141458
129,Gonzalo Melero,Levante,0.044751,0.0,2.0,0.040850,0.125602
74,Tete Morente,Elche,0.036938,1.0,2.0,0.027243,0.124181
92,Pedri,Barcelona,0.042433,1.0,1.0,0.040850,0.123283
0,Luis Rioja,Alavés,0.035524,1.0,1.0,0.041005,0.116529


In [19]:
#Exportamos a csv
df_lm.to_csv('/Users/alexcar934/Desktop/EDA_bien/src/data/processed/csv_principales/6_Banda_Izquierda.csv')